In [ ]:
import json
import os
from apify_client import ApifyClient
from dotenv import load_dotenv

# Initialize the Apify client with your API token
load_dotenv()
APIFY_TOKEN = os.getenv("APIFY_TOKEN")
client = ApifyClient(APIFY_TOKEN)

"""_summary_
Reduces each dataset item to only the relevant fields.
"""
def slim_down_listing(raw: dict) -> dict:
    return {
        "id":             raw.get("id"),
        "coordinates":    raw.get("coordinates"),
        "propertyType":   raw.get("propertyType"),
        "roomType":       raw.get("roomType"),
        "homeTier":       raw.get("homeTier"),
        "personCapacity": raw.get("personCapacity"),
        "rating":         raw.get("rating"),
        "title":          raw.get("title"),
        "subDescription": raw.get("subDescription"),
        "amenities":      raw.get("amenities"),
    }

"""_summary_
Reads Airbnb room IDs, builds URLs, runs the Apify Actor, and extracts selected fields.
"""
def main():
    # Read the Airbnb listing IDs from 'ids.txt' and cnvert to Airbnb URL
    with open("ids.txt", "r", encoding="utf-8") as f:
        ids = [line.strip() for line in f if line.strip()]
    room_urls = [f"https://www.airbnb.mx/rooms/{room_id}" for room_id in ids]

    # Prepare the input for the actor
    run_input = {
        "startUrls": [{"url": url} for url in room_urls],
    }
    run = client.actor("tri_angle~airbnb-rooms-urls-scraper").call(run_input=run_input)

    # Once completed, access the dataset to retrieve the scraped data
    dataset_id = run["defaultDatasetId"]
    all_listings = []
    for item in client.dataset(dataset_id).iterate_items():
        slim = slim_down_listing(item)
        all_listings.append(slim)

    # Save all reduced listings into a JSON file
    with open("all_listings.json", "w", encoding="utf-8") as out_f:
        json.dump(all_listings, out_f, ensure_ascii=False, indent=2)
    print(f"Process complete. Saved {len(all_listings)} records into all_listings.json.")

if __name__ == "__main__":
    main()

Process complete. Saved 2 records into all_listings.json.
